In [1]:
import os
import numpy as np

import pandas as pd
import tensorflow as tf

from transformers import BertTokenizer, RobertaTokenizer
from Trainer import Trainer
from preprocessing import dataPreprocessor
from parameters import *
from RoBERTaModel import *

from pathlib import Path

import horovod.tensorflow as hvd

tf.__version__

'2.1.0'

In [2]:
        hvd.init()
        gpus = tf.config.list_physical_devices('GPU') 
        for gpu in gpus:
            print(gpu)
            tf.config.experimental.set_memory_growth(gpu, True)
        if gpus:
            print("gpus ", gpus)
            print("local rank ",hvd.local_rank())
            tf.config.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
            print(tf.config.get_visible_devices())

In [3]:
data_dir = "google-quest-challenge/"

train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))
submit_df = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
stack_df = pd.read_csv(os.path.join(data_dir, "stackexchange_data.csv"))

In [4]:
input_df = train_df[train_columns + target_columns]

In [5]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

dataPreprocessor.logger = False
dataPreprocessor.tokenizer = tokenizer
dataPreprocessor.model = "Roberta"

In [6]:
model_name = "RoBERTaForQALabeling"

checkpoint_dir = os.path.join(save_dir, "{}_tokenizer_data" .format(model_name))
Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
print("saving tokenizer in ", checkpoint_dir)
tokenizer.save_pretrained(checkpoint_dir)

saving tokenizer in  ./checkpoints/RoBERTaForQALabeling_tokenizer_data


('./checkpoints/RoBERTaForQALabeling_tokenizer_data/vocab.json',
 './checkpoints/RoBERTaForQALabeling_tokenizer_data/merges.txt',
 './checkpoints/RoBERTaForQALabeling_tokenizer_data/special_tokens_map.json',
 './checkpoints/RoBERTaForQALabeling_tokenizer_data/added_tokens.json')

In [7]:
Trainer.train(model_name=model_name,
              tokenizer=tokenizer,
              input_df=input_df[:30])

Fold1/5 
Preprocessing train data
Preprocessing test data


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-33f3aa9bf593>", line 3, in <module>
    input_df=input_df[:30])
  File "/shared/ML/KaggleQA_Labeling/Trainer.py", line 84, in train
    model = models[model_name][0].from_pretrained(weights, num_labels=num_labels, output_hidden_states=True)
  File "/usr/local/lib/python3.6/dist-packages/transformers/modeling_tf_utils.py", line 314, in from_pretrained
    proxies=proxies,
  File "/usr/local/lib/python3.6/dist-packages/transformers/file_utils.py", line 254, in cached_path
    local_files_only=local_files_only,
  File "/usr/local/lib/python3.6/dist-packages/transformers/file_utils.py", line 487, in get_from_cache
    http_get(url, temp_file, proxies=proxies, resume_size=resume_size, user_agent=user_agent)
  File "/usr/local/lib/python3.6/dist-packages/transformers/file_util

KeyboardInterrupt: 

In [ ]:
Trainer.pseudo_predict(model_name=model_name, 
                       pseudo_df=stack_df[:40])

In [179]:
print("CLS ", tokenizer.cls_token_id)
print("SEP ", tokenizer.sep_token_id)
print("PAD ", tokenizer.pad_token_id)
print("MASK", tokenizer.mask_token_id)

inputs = [0, 3,4,5,65,76,787,2 ,98,856,34,0, 543,654,15, 245, 5678, 435, 1, 1, 1, 1]

vocab_without_tokens = []

aaa = tokenizer.encode("Fight", add_special_tokens=False)
print(aaa)
print(tokenizer.decode(aaa))

tokenizer_vocab = tokenizer.get_vocab().keys() - tokenizer.special_tokens_map.values()

for key in tokenizer_vocab:
    if key != tokenizer.cls_token and key != tokenizer.sep_token and key != tokenizer.pad_token:
        vocab_without_tokens.append(key)

        
def mask_tokens(inputs, tokenizer, tokenizer_vocab):        

    feasible_indexes = []
    
    # not masking special tokens
    for idx, token in enumerate(inputs):
        if token != tokenizer.cls_token_id and token != tokenizer.sep_token_id and token != tokenizer.pad_token_id:
            feasible_indexes.append(idx)

    
    masked_input = copy.copy(inputs)
    
    #shuffle randmly and get 15% of data

    np.random.shuffle(feasible_indexes)
    desired_change_count = max(1, int(len(feasible_indexes)*0.15))

    changed_amount = 0

    for idx in feasible_indexes:
        # if we changed desired amount of tokens break
        if changed_amount == desired_change_count:
            break

        # 80% to mask token
        if np.random.random() < 0.8:
            masked_input[idx] = tokenizer.mask_token_id
            changed_amount += 1
            continue

        # 10% to change it with random word from vocabulary
        # because ther eare only 20% left we have two options:
        # random word
        # leave it a it was
        if np.random.random() < 0.5:
            masked_input[idx] = np.random.randint(0, len(tokenizer_vocab))
            changed_amount += 1
            continue
        # else 10% chance to leave it as it was
        else:
            masked_input[idx] = masked_input[idx]
            continue

    return masked_input

CLS  0
SEP  2
PAD  1
MASK 50264
[40520]
Fight


In [196]:
output = mask_tokens(inputs, tokenizer, vocab_without_tokens)

In [197]:
print(output)

[0, 3, 4, 5, 65, 76, 787, 2, 98, 856, 34, 0, 50264, 654, 13898, 245, 5678, 435, 1, 1, 1, 1]


In [141]:
tokenizer.special_tokens_map.values()

dict_values(['<s>', '</s>', '<unk>', '</s>', '<pad>', '<s>', '<mask>'])

In [15]:
a = tf.Variable([[1,2,3,4,5,6,76,8]], dtype=tf.int64)
print(a.shape)

(1, 8)


In [20]:
b = tf.concat([a[:, :3],[[54]],a[:, 3+1:]], axis = -1)
print(b)

tf.Tensor([[ 1  2  3 54  5  6 76  8]], shape=(1, 8), dtype=int64)
